In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")
# options.add_argument("--headless")  # Enable headless mode for faster scraping
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver_path = r"C:\Users\New\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
driver = webdriver.Chrome(service=Service(driver_path), options=options)

# Navigate to Twitter login page
driver.get("https://twitter.com/login")
time.sleep(30)  # Wait for manual login

# Input hashtag and max tweets
hashtag = input("Enter the hashtag to search for (include #): ").strip()
max_tweets = int(input("Enter the maximum number of tweets to extract: "))

# Navigate to Twitter's search page
url = f"https://twitter.com/search?q={hashtag}&src=typed_query&f=live"
driver.get(url)
time.sleep(10)  # Allow initial tweets to load

# Scrape tweets
tweets_data = []
last_height = driver.execute_script("return document.body.scrollHeight")

while len(tweets_data) < max_tweets:
    tweets = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    
    for tweet in tweets:
        try:
            content = tweet.find_element(By.XPATH, './/div[@lang]').text
        except:
            content = None
        try:
            username = tweet.find_element(By.XPATH, './/span[contains(text(), "@")]').text
        except:
            username = None
        try:
            date = tweet.find_element(By.XPATH, './/time').get_attribute("datetime")
        except:
            date = None
        try:
            retweets = tweet.find_element(By.XPATH, './/button[@data-testid="retweet"]/div[@dir="ltr"]/div/span/span').text
        except:
            retweets = "0"
        try:
            comments = tweet.find_element(By.XPATH, './/button[@data-testid="reply"]/div[@dir="ltr"]/div/span/span').text 

        except:
            comments = "0"
        try:
            likes = tweet.find_element(By.XPATH, './/button[@data-testid="like"]/div[@dir="ltr"]/div/span/span').text
        except:
            likes = "0"

        try:
            views = tweet.find_element(By.XPATH, './/a[contains(@href, "/analytics")]').get_attribute("aria-label")
        except:
            views = "0 Views"
        try:
            images = tweet.find_elements(By.XPATH, './/img[contains(@src, "twimg")]')
            image_urls = [img.get_attribute("src") for img in images]
        except:
            image_urls = []
        try:
            hashtags = [tag.text for tag in tweet.find_elements(By.XPATH, './/a[contains(@href, "/hashtag/")]')]
        except:
            hashtags = []
        try:
            is_verified = bool(tweet.find_element(By.XPATH, './/svg[@aria-label="Verified account"]'))
        except:
            is_verified = False

        tweet_data = {
            "user_posted": username,
            "date_posted": date,
            "description": content,
            "views": views,
            "replies": comments,
            "reposts": retweets,
            "likes": likes,
            "hashtags": ", ".join(hashtags),
            "is_verified": is_verified,
            "photos": ", ".join(image_urls),
            "url": url
        }

        if tweet_data not in tweets_data:
            tweets_data.append(tweet_data)
        if len(tweets_data) >= max_tweets:
            break

    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:  # End scrolling if no new content
        break
    last_height = new_height

# Close the browser
driver.quit()

# Save tweets to CSV
if tweets_data:
    output_file = f"{hashtag[1:]}_tweets.csv"
    df = pd.DataFrame(tweets_data)
    df.to_csv(output_file, index=False)
    print(f"Tweets saved to {output_file}")
else:
    print("No tweets found.")


Enter the hashtag to search for (include #):  #chennai
Enter the maximum number of tweets to extract:  400


Tweets saved to chennai_tweets.csv
